In [4]:
import pandas as pd
import os

# Read in the dataset
trans_df = pd.read_csv(os.path.join("..", "data", "Final with Coords, Fuel Type and Prices.csv"))
vehicle_df = pd.read_csv(os.path.join("..", "data", "Final KMPL dataset.csv"))

In [5]:
trans_df.shape

(175933, 21)

In [6]:
vehicle_df.columns

Index(['Reg', 'Overlap_Start', 'Overlap_End', 'Total_Km', 'Total_Litres',
       'KMPL', 'REG_NUM', 'VEHICLE MAKE', 'MODEL DERIVATIVE', 'DEPARTMENT',
       'District', 'Site', 'Site Lat', 'Site Long', 'Fuel Type',
       'RATE CARD CATEGORY'],
      dtype='object')

In [7]:
trans_df.columns

Index(['Transaction Date', 'REG_NUM', 'Merchant Name', 'No. of Litres',
       'Transaction Amount', 'VEHICLE MAKE', 'MODEL DERIVATIVE', 'DEPARTMENT',
       'RATE CARD CATEGORY', 'Site', 'District', 'Site Lat', 'Site Long',
       'Merchant Lat', 'Merchant Long', 'Fuel Type',
       'Estimated Price Per Litre', 'Coastal Petrol', 'Inland Petrol',
       'Coastal Diesel', 'Inland Diesel'],
      dtype='object')

In [8]:
# Remove the word "CATEGORY" from the 'RATE CARD CATEGORY' in vehicle_df
vehicle_df['RATE CARD CATEGORY'] = vehicle_df['RATE CARD CATEGORY'].str.replace('CATEGORY', '')

trans_df['RATE CARD CATEGORY'] = trans_df['RATE CARD CATEGORY'].str.replace('CATEGORY', '')

# Fleet Composition

In [55]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import FixedLocator, FixedFormatter

def shorten_names(names, max_length=20):
    shortened_names = []
    for name in names:
        if len(name) > max_length:
            shortened_name = name[:max_length-3] + '...'
        else:
            shortened_name = name
        shortened_names.append(shortened_name)
    return shortened_names


def create_countplot(data, filename, threshold, max_length=50):
    # Filter the data and calculate the sum of counts for the remaining categories
    filtered_data = data[data >= threshold]
    others_count = data[data < threshold].sum()
    filtered_data['Others'] = others_count

    # Shorten the names for each category
    shortened_names = shorten_names(filtered_data.index, max_length=max_length)

    # Create a single plot
    fig, ax = plt.subplots(figsize=(8, 8))

    # Set the font size for the title and labels
    label_font_size = 12
    y_label_font_size = 14

    # Plot the data
    ax.bar(shortened_names, filtered_data)
    ax.set_xticklabels(shortened_names, rotation=45, ha='right', fontsize=label_font_size)
    ax.set_yticklabels(ax.get_yticks().astype(int), fontsize=label_font_size)
    ax.set_ylabel('Count', fontsize=y_label_font_size)

    # Adjust the spacing
    plt.tight_layout()

    # Save the plot as a PDF file
    plt.savefig(filename, format='pdf', bbox_inches='tight')

    # Close the plot
    plt.close(fig)


def countplot_side_by_side(data1, data2, title1, title2, filename, threshold1, threshold2, max_length=50):
    # Filter the data and calculate the sum of counts for the remaining categories
    filtered_data1 = data1[data1 >= threshold1]
    others_count1 = data1[data1 < threshold1].sum()
    filtered_data1 = filtered_data1.to_frame()

    if others_count1 > 0:
        filtered_data1.loc['Others'] = others_count1

    filtered_data2 = data2[data2 >= threshold2]
    others_count2 = data2[data2 < threshold2].sum()
    filtered_data2 = filtered_data2.to_frame()

    if others_count2 > 0:
        filtered_data2.loc['Others'] = others_count2

    # Shorten the names for each category
    shortened_names1 = shorten_names(filtered_data1.index, max_length=max_length)
    shortened_names2 = shorten_names(filtered_data2.index, max_length=max_length)

    # Create a figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

    # Set the font size for the labels
    label_font_size = 12
    y_label_font_size = 14

    # Plot the data for the first subplot
    ax1.bar(shortened_names1, filtered_data1.iloc[:, 0])
    ax1.set_xticklabels(shortened_names1, rotation=45, ha='right', fontsize=label_font_size)
    ax1.set_ylabel('Count', fontsize=y_label_font_size)
    ax1.set_title(f'a) {title1}')

    # Plot the data for the second subplot
    ax2.bar(shortened_names2, filtered_data2.iloc[:, 0])
    ax2.set_xticklabels(shortened_names2, rotation=45, ha='right', fontsize=label_font_size)
    ax2.set_title(f'b) {title2}')

    # Set the y-axis ticks and labels to integer values for both subplots
    yticks1 = ax1.get_yticks().astype(int)
    yticks2 = ax2.get_yticks().astype(int)
    ax1.yaxis.set_major_locator(FixedLocator(yticks1))
    ax1.yaxis.set_major_formatter(FixedFormatter(yticks1))
    ax1.tick_params(axis='y', labelsize=label_font_size)
    ax2.yaxis.set_major_locator(FixedLocator(yticks2))
    ax2.yaxis.set_major_formatter(FixedFormatter(yticks2))
    ax2.tick_params(axis='y', labelsize=label_font_size)

    # Adjust the spacing
    plt.tight_layout()

    # Save the plot as a PDF file
    plt.savefig(filename, format='pdf', bbox_inches='tight')

    # Close the plot
    plt.close(fig)


def histogram_side_by_side(data1, var1, data2, var2, title1, title2, filename, bins=20):
    # Create a figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

    # Set the font size for the labels
    label_font_size = 12
    y_label_font_size = 14

    # Plot the data for the first subplot
    ax1.hist(data1[var1], bins=bins, edgecolor='black')
    ax1.set_xlabel(var1, fontsize=label_font_size)
    ax1.set_ylabel('Frequency', fontsize=y_label_font_size)
    ax1.set_title(f'a) {title1}')

    # Plot the data for the second subplot
    ax2.hist(data2[var2], bins=bins, edgecolor='black')
    ax2.set_xlabel(var2, fontsize=label_font_size)
    ax2.set_ylabel('Frequency', fontsize=y_label_font_size)
    ax2.set_title(f'b) {title2}')

    # Set the y-axis ticks and labels to integer values for both subplots
    yticks1 = ax1.get_yticks().astype(int)
    yticks2 = ax2.get_yticks().astype(int)
    ax1.yaxis.set_major_locator(FixedLocator(yticks1))
    ax1.yaxis.set_major_formatter(FixedFormatter(yticks1))
    ax1.tick_params(axis='y', labelsize=label_font_size)
    ax2.yaxis.set_major_locator(FixedLocator(yticks2))
    ax2.yaxis.set_major_formatter(FixedFormatter(yticks2))
    ax2.tick_params(axis='y', labelsize=label_font_size)

    # Adjust the spacing
    plt.tight_layout()

    # Save the plot as a PDF file
    plt.savefig(filename, format='pdf', bbox_inches='tight')

    # Close the plot
    plt.close(fig)

def barplot_side_by_side(data, cat_var, cont_var1, cont_var2, title1, title2, filename, max_length=50):
    # Group the data by the categorical variable and calculate the mean of the continuous variables
    grouped_data1 = data.groupby(cat_var)[cont_var1].mean().nlargest(9)
    grouped_data2 = data.groupby(cat_var)[cont_var2].mean().nlargest(9)

    # Calculate the sum of the remaining means for the "Other" category
    other_mean1 = data[~data[cat_var].isin(grouped_data1.index)].groupby(cat_var)[cont_var1].mean().mean()
    other_mean2 = data[~data[cat_var].isin(grouped_data2.index)].groupby(cat_var)[cont_var2].mean().mean()

    # Append the "Other" category to the grouped data
    grouped_data1['Other'] = other_mean1
    grouped_data2['Other'] = other_mean2

    # Shorten the names for each category
    shortened_names = shorten_names(grouped_data1.index, max_length=max_length)

    # Create a figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6))

    # Set the font size for the labels
    label_font_size = 12
    y_label_font_size = 14

    # Plot the data for the first subplot
    ax1.bar(shortened_names, grouped_data1)
    ax1.set_xticklabels(shortened_names, rotation=45, ha='right', fontsize=label_font_size)
    ax1.set_ylabel(f'Average {cont_var1}', fontsize=y_label_font_size)
    ax1.set_title(f'a) {title1}')

    # Plot the data for the second subplot
    ax2.bar(shortened_names, grouped_data2)
    ax2.set_xticklabels(shortened_names, rotation=45, ha='right', fontsize=label_font_size)
    ax2.set_ylabel(f'Average {cont_var2}', fontsize=y_label_font_size)
    ax2.set_title(f'b) {title2}')

    # Adjust the spacing
    plt.tight_layout()

    # Save the plot as a PDF file
    plt.savefig(filename, format='pdf', bbox_inches='tight')

    # Close the plot
    plt.close(fig)

In [11]:
# Group the DataFrame by 'MODEL DERIVATIVE' and count the occurrences
fleet_data_derivative = vehicle_df['MODEL DERIVATIVE'].value_counts()
trans_data_derivative = trans_df['MODEL DERIVATIVE'].value_counts()

# Group the DataFrame by 'VEHICLE MAKE' and count the occurrences
fleet_data_make = vehicle_df['VEHICLE MAKE'].value_counts()
trans_data_make = trans_df['VEHICLE MAKE'].value_counts()

# Group the DataFrame by 'DEPARTMENT' and count the occurrences
fleet_data_department = vehicle_df['DEPARTMENT'].value_counts()
trans_data_department = trans_df['DEPARTMENT'].value_counts()

# Group the DataFrame by 'RATE CARD CATEGORY' and count the occurrences
fleet_data_rate_card = vehicle_df['RATE CARD CATEGORY'].value_counts()
trans_data_rate_card = trans_df['RATE CARD CATEGORY'].value_counts()

In [17]:
# Create and save individual plots
countplot_side_by_side(fleet_data_derivative, trans_data_derivative, 
                     'Fleet', 'Transactions', 
                     '../plots/eda/model_derivative.pdf', 
                     20, 5000,
                     20)

countplot_side_by_side(fleet_data_make, trans_data_make, 
                     'Fleet', 'Transactions', 
                     '../plots/eda/make.pdf', 
                     20, 5000,
                     20)

countplot_side_by_side(fleet_data_department, trans_data_department, 
                     'Fleet', 'Transactions', 
                     '../plots/eda/department.pdf', 
                     20, 5000,
                     20)

countplot_side_by_side(fleet_data_rate_card, trans_data_rate_card, 
                     'Fleet', 'Transactions', 
                     '../plots/eda/rate_card.pdf', 
                     20, 5000,
                     20)

/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/198997191.py:77: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(shortened_names1, rotation=45, ha='right', fontsize=label_font_size)
/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/198997191.py:83: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax2.set_xticklabels(shortened_names2, rotation=45, ha='right', fontsize=label_font_size)
/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/198997191.py:77: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(shortened_names1, rotation=45, ha='right', fontsize=label_font_size)
/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/198997191.py:83: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax2.set_xticklabels(shortened_names2, rotation=45, ha='right', fontsize=label_font_size)
/var/fol

In [13]:
# Convert the date column to a datetime object
trans_df['Transaction Date'] = pd.to_datetime(trans_df['Transaction Date'])

# Create a new column for the month name
trans_df['Month Name'] = trans_df['Transaction Date'].dt.month_name()

# Create a new column for the weekday name
trans_df['Weekday Name'] = trans_df['Transaction Date'].dt.day_name()

In [14]:
trans_df.columns

Index(['Transaction Date', 'REG_NUM', 'Merchant Name', 'No. of Litres',
       'Transaction Amount', 'VEHICLE MAKE', 'MODEL DERIVATIVE', 'DEPARTMENT',
       'RATE CARD CATEGORY', 'Site', 'District', 'Site Lat', 'Site Long',
       'Merchant Lat', 'Merchant Long', 'Fuel Type',
       'Estimated Price Per Litre', 'Coastal Petrol', 'Inland Petrol',
       'Coastal Diesel', 'Inland Diesel', 'Month Name', 'Weekday Name'],
      dtype='object')

In [15]:
trans_data_wday = trans_df['Weekday Name'].value_counts()
trans_data_month = trans_df['Month Name'].value_counts()

In [16]:
# Create and save individual plots
countplot_side_by_side(trans_data_wday, trans_data_month, 
                     'Weekday', 'Month', 
                     '../plots/eda/month_weekday.pdf', 
                     5000, 5000,
                     30)

/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/198997191.py:77: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(shortened_names1, rotation=45, ha='right', fontsize=label_font_size)
/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/198997191.py:83: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax2.set_xticklabels(shortened_names2, rotation=45, ha='right', fontsize=label_font_size)


In [37]:
# Rename the "UNKNOWN" category to "Unknown" in the 'District' column
trans_df['District'] = trans_df['District'].replace('UNKNOWN', 'Unknown')
trans_df['Site'] = trans_df['Site'].replace('Pe', 'PE')

# Remove the values where Distict is "Unknown"
trans_df_nd = trans_df[trans_df['District'] != "Unknown"]
trans_df_ns = trans_df[trans_df['Site'] != "Unknown"]

trans_data_district = trans_df_nd['District'].value_counts()
trans_data_site = trans_df_ns['Site'].value_counts()

In [44]:
# Create and save individual plots
countplot_side_by_side(trans_data_district, trans_data_site, 
                     'District', 'Site', 
                     '../plots/eda/district_site.pdf', 
                     3000, 1800,
                     30)

/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/731920365.py:77: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(shortened_names1, rotation=45, ha='right', fontsize=label_font_size)
/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/731920365.py:83: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax2.set_xticklabels(shortened_names2, rotation=45, ha='right', fontsize=label_font_size)


In [27]:
# Filter out all transaction amounts that are greater than 5000
trans_df_ne = trans_df[trans_df['Transaction Amount'] <= 5000]
trans_df_ne = trans_df_ne[trans_df_ne['Transaction Amount'] > 0]

trans_df_ne = trans_df_ne[trans_df_ne['No. of Litres'] <= 200]

histogram_side_by_side(trans_df_ne, 'Transaction Amount', trans_df_ne, 'No. of Litres',
                       'Transaction Amount', 'Number of Litres', 
                     '../plots/eda/transaction_amount.pdf', 
                     20)

## Transaction and fleet amount means across different vars

In [57]:
trans_df.columns

Index(['Transaction Date', 'REG_NUM', 'Merchant Name', 'No. of Litres',
       'Transaction Amount', 'VEHICLE MAKE', 'MODEL DERIVATIVE', 'DEPARTMENT',
       'RATE CARD CATEGORY', 'Site', 'District', 'Site Lat', 'Site Long',
       'Merchant Lat', 'Merchant Long', 'Fuel Type',
       'Estimated Price Per Litre', 'Coastal Petrol', 'Inland Petrol',
       'Coastal Diesel', 'Inland Diesel', 'Month Name', 'Weekday Name'],
      dtype='object')

In [58]:
barplot_side_by_side(trans_df, 'VEHICLE MAKE', 'Transaction Amount', 'No. of Litres',
                     'Transaction Amount', 'Number of Litres', 
                     '../plots/eda/biplot_make.pdf', 
                     25)

barplot_side_by_side(trans_df, 'MODEL DERIVATIVE', 'Transaction Amount', 'No. of Litres',
                     'Transaction Amount', 'Number of Litres', 
                     '../plots/eda/biplot_derivative.pdf', 
                     25)

barplot_side_by_side(trans_df, 'DEPARTMENT', 'Transaction Amount', 'No. of Litres',
                     'Transaction Amount', 'Number of Litres', 
                     '../plots/eda/biplot_department.pdf', 
                     25)

barplot_side_by_side(trans_df, 'RATE CARD CATEGORY', 'Transaction Amount', 'No. of Litres',
                     'Transaction Amount', 'Number of Litres', 
                     '../plots/eda/biplot_category.pdf', 
                     25)

barplot_side_by_side(trans_df, 'District', 'Transaction Amount', 'No. of Litres',
                     'Transaction Amount', 'Number of Litres', 
                     '../plots/eda/biplot_district.pdf', 
                     25)

barplot_side_by_side(trans_df, 'Month Name', 'Transaction Amount', 'No. of Litres',
                     'Transaction Amount', 'Number of Litres', 
                     '../plots/eda/biplot_month.pdf', 
                     25)

barplot_side_by_side(trans_df, 'Weekday Name', 'Transaction Amount', 'No. of Litres',
                     'Transaction Amount', 'Number of Litres', 
                     '../plots/eda/biplot_wday.pdf', 
                     25)

/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/3085069622.py:170: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(shortened_names, rotation=45, ha='right', fontsize=label_font_size)
/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/3085069622.py:176: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax2.set_xticklabels(shortened_names, rotation=45, ha='right', fontsize=label_font_size)
/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/3085069622.py:170: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(shortened_names, rotation=45, ha='right', fontsize=label_font_size)
/var/folders/v7/l0jq1cbd0fnf4ytmwzksxt0r0000gn/T/ipykernel_64108/3085069622.py:176: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax2.set_xticklabels(shortened_names, rotation=45, ha='right', fontsize=label_font_size)
/var

In [59]:
# Save the filtered dataset to a new CSV file
trans_df.to_csv(os.path.join("..", "data", "Final for clustering.csv"), index=False)